# ✎ HuggingFace Datasets Hub Integration

## Overview

> Prerequisite: make sure that you have installed fairseq2 with `pip install fairseq2[arrow]`. This will install additional packages required for parquet dataloader (_e.g._ pyarrow, retrying, polars, xxhash).


In [1]:
from fairseq2.data.parquet.fragment_streaming import (
    FragmentStreamingConfig, ParquetFragmentStreamer
)
from fairseq2.data.parquet.fragment_loading import (
    FragmentLoadingConfig, ParquetFragmentLoader
)

from huggingface_hub import HfFileSystem
hf_fs = HfFileSystem()  # init FileSystem interface for HF

In [2]:
source_dataset_glob_path = "datasets/cais/mmlu/*/*.parquet"
all_paths = hf_fs.glob(source_dataset_glob_path)  # taking all parquet files
test_paths = [path for path in all_paths if "test-" in path]  # filtering manually the path since there's no proper hive partitionning
test_paths[:10]

['datasets/cais/mmlu/abstract_algebra/test-00000-of-00001.parquet',
 'datasets/cais/mmlu/all/test-00000-of-00001.parquet',
 'datasets/cais/mmlu/anatomy/test-00000-of-00001.parquet',
 'datasets/cais/mmlu/astronomy/test-00000-of-00001.parquet',
 'datasets/cais/mmlu/business_ethics/test-00000-of-00001.parquet',
 'datasets/cais/mmlu/clinical_knowledge/test-00000-of-00001.parquet',
 'datasets/cais/mmlu/college_biology/test-00000-of-00001.parquet',
 'datasets/cais/mmlu/college_chemistry/test-00000-of-00001.parquet',
 'datasets/cais/mmlu/college_computer_science/test-00000-of-00001.parquet',
 'datasets/cais/mmlu/college_mathematics/test-00000-of-00001.parquet']

In [3]:
fragment_config = FragmentStreamingConfig(
    parquet_path=test_paths,
    nb_epochs=1,
    filesystem=hf_fs,
    split_to_row_groups=True,
    fragment_shuffle_window=0,  # no shuffling
    files_circular_shift=False,
)

streamer = ParquetFragmentStreamer(config=fragment_config)

# Create the loading config
loading_config = FragmentLoadingConfig(
    columns=None,
    cache=False,
    rename_columns=False,
    add_fragment_traces=False,  # keeping only original columns
    drop_null=False,  # Keep null values
    nb_prefetch=1,  # Prefetch 2 fragments
    num_parallel_fragments=4,  # Process 4 fragments in parallel
    filters='pc.field("answer") == 0',  # A dummy filter to select only samples with answer=0
)

# Build the loading pipeline
loader = ParquetFragmentLoader(config=loading_config)

fragment_pipeline = streamer.build_pipeline(0, 1)
loading_pipeline = loader.apply(fragment_pipeline)
result = list(iter(loading_pipeline.and_return()))

In [4]:
len(result)

73

In [5]:
print([len(x) for x in result])

[22, 220, 242, 208, 187, 211, 208, 264, 244, 255, 214, 230, 240, 239, 253, 7, 25, 27, 30, 57, 37, 20, 26, 21, 36, 22, 28, 62, 27, 35, 79, 36, 18, 55, 31, 25, 36, 35, 38, 79, 57, 50, 30, 105, 33, 51, 64, 70, 34, 29, 28, 36, 35, 18, 68, 30, 186, 86, 213, 69, 58, 70, 66, 246, 131, 50, 153, 24, 46, 49, 28, 47, 55]


In [6]:
# preview the result with Polars
import polars as pl
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_fmt_table_cell_list_len(200)
pl.from_arrow(result[0])

shape: (22, 4)
┌───────────────────────────────────┬──────────────────┬──────────────────────────────────┬────────┐
│ question                          ┆ subject          ┆ choices                          ┆ answer │
│ ---                               ┆ ---              ┆ ---                              ┆ ---    │
│ str                               ┆ str              ┆ list[str]                        ┆ i64    │
╞═══════════════════════════════════╪══════════════════╪══════════════════════════════════╪════════╡
│ Statement 1 | If a group has an   ┆ abstract_algebra ┆ ["True, True", "False, False",   ┆ 0      │
│ element of order 15 it must have  ┆                  ┆ "True, False", "False, True"]    ┆        │
│ at least 8 elements of order 15.  ┆                  ┆                                  ┆        │
│ St…                               ┆                  ┆                                  ┆        │
│ Statement 1 | Every homomorphic   ┆ abstract_algebra ┆ ["True, True", "False, False",   ┆ 0      │
│ image of a group G is isomorphic  ┆                  ┆ "True, False", "False, True"]    ┆        │
│ to a factor group of G. Statement ┆                  ┆                                  ┆        │
│ 2…                                ┆                  ┆                                  ┆        │
│ Find the order of the factor      ┆ abstract_algebra ┆ ["1", "2", "5", "11"]            ┆ 0      │
│ group (Z_11 x Z_15)/(<1, 1>)      ┆                  ┆                                  ┆        │
│ Statement 1| Every group of order ┆ abstract_algebra ┆ ["True, True", "False, False",   ┆ 0      │
│ p^2 where p is prime is Abelian.  ┆                  ┆ "True, False", "False, True"]    ┆        │
│ Statement 2 | For a fixed prime   ┆                  ┆                                  ┆        │
│ p…                                ┆                  ┆                                  ┆        │
│ Statement 1 | For finite groups G ┆ abstract_algebra ┆ ["True, True", "False, False",   ┆ 0      │
│ and H, |G + H| = |G||H|. (G + H   ┆                  ┆ "True, False", "False, True"]    ┆        │
│ is the internal direct product.)  ┆                  ┆                                  ┆        │
│ S…                                ┆                  ┆                                  ┆        │
│ …                                 ┆ …                ┆ …                                ┆ …      │
│ Statement 1 | For every positive  ┆ abstract_algebra ┆ ["True, True", "False, False",   ┆ 0      │
│ integer n there is a cyclic group ┆                  ┆ "True, False", "False, True"]    ┆        │
│ of order n. Statement 2 | Every   ┆                  ┆                                  ┆        │
│ f…                                ┆                  ┆                                  ┆        │
│ Statement 1 | If H and K are      ┆ abstract_algebra ┆ ["True, True", "False, False",   ┆ 0      │
│ subgroups of a group G, then |HK| ┆                  ┆ "True, False", "False, True"]    ┆        │
│ = |H||K|/|H intersection K|.      ┆                  ┆                                  ┆        │
│ Statemen…                         ┆                  ┆                                  ┆        │
│ For T: Z x Z -> Z where T(1, 0) = ┆ abstract_algebra ┆ ["-19", "-10", "19", "10"]       ┆ 0      │
│ 3 and T(0, 1) = -5, find T(-3,2). ┆                  ┆                                  ┆        │
│ Compute the product in the given  ┆ abstract_algebra ┆ ["0", "1", "4", "6"]             ┆ 0      │
│ ring. (12)(16) in Z_24            ┆                  ┆                                  ┆        │
│ Find the maximum possible order   ┆ abstract_algebra ┆ ["6", "12", "30", "105"]         ┆ 0      │
│ for an element of S_n for n = 6.  ┆                  ┆                                  ┆        │
└───────────────────────────────────┴──────────────────┴──────────────────────────────────┴────────┘